<a href="https://colab.research.google.com/github/Arjun8242/Fake_News_Detection_model/blob/main/Fake_News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [2]:

import pandas as pd
import numpy as np
import nltk

In [ ]:
fake_df=pd.read_csv('/content/drive/MyDrive/Fake.csv')
true_df=pd.read_csv('/content/drive/MyDrive/True.csv')
print("True News Examples:")
print(true_df.head())

print("\nFake News Examples:")
print(fake_df.head())

True News Examples:
                                               title  \
0  As U.S. budget fight looms, Republicans flip t...   
1  U.S. military to accept transgender recruits o...   
2  Senior U.S. Republican senator: 'Let Mr. Muell...   
3  FBI Russia probe helped by Australian diplomat...   
4  Trump wants Postal Service to charge 'much mor...   

                                                text       subject  \
0  WASHINGTON (Reuters) - The head of a conservat...  politicsNews   
1  WASHINGTON (Reuters) - Transgender people will...  politicsNews   
2  WASHINGTON (Reuters) - The special counsel inv...  politicsNews   
3  WASHINGTON (Reuters) - Trump campaign adviser ...  politicsNews   
4  SEATTLE/WASHINGTON (Reuters) - President Donal...  politicsNews   

                 date  
0  December 31, 2017   
1  December 29, 2017   
2  December 31, 2017   
3  December 30, 2017   
4  December 29, 2017   

Fake News Examples:
                                               title  \
0

In [ ]:
print("True news shape:", true_df.shape)
print("Fake news shape:", fake_df.shape)

True news shape: (21417, 4)
Fake news shape: (23481, 4)


In [ ]:
true_df['label']=1
fake_df['label']=0

In [ ]:
df=pd.concat([true_df,fake_df],ignore_index=True)
df=df.sample(frac=1, random_state=42).reset_index(drop=True)
df.to_csv('news.csv', index=False)

In [ ]:
df.describe()

,label
count,44898.000000
mean,0.477015
std,0.499477
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   label    44898 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.7+ MB


In [ ]:
df.isnull().sum()
if 'content' not in df.columns:
    df['content'] = df['title'] + " " + df['text']

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import string
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words=set(stopwords.words('english'))
stemmer=PorterStemmer()
def preprocess_text(text):
    text=text.translate(str.maketrans('','', string.punctuation))
    text=text.lower()
    text=''.join([char for char in text if not char.isdigit()])
    text=' '.join([word for word in text.split() if word not in stop_words])
    text=' '.join([stemmer.stem(word) for word in text.split()])
    return text
df['content']=df['content'].apply(preprocess_text)

In [ ]:
from sklearn.model_selection import train_test_split
X=df['content']
Y=df['label']
X_train, X_test, Y_train, Y_test = train_test_split(df['content'], df['label'], test_size=0.2, random_state=42)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer()
X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

In [ ]:
from sklearn.svm import LinearSVC
model = LinearSVC()
model.fit(X_train, Y_train)


LinearSVC()

In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
Y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(Y_test, Y_pred))
print("Classification Report:\n", classification_report(Y_test, Y_pred))
print("Confusion Matrix:\n", confusion_matrix(Y_test, Y_pred))

Accuracy: 0.9964365256124722
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      4669
           1       1.00      1.00      1.00      4311

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980

Confusion Matrix:
 [[4648   21]
 [  11 4300]]


In [ ]:
sample = ["In March 2020, India reported over 1 million confirmed COVID-19 cases, according to WHO."]
sample_tfidf = vectorizer.transform(sample)
print(model.predict(sample_tfidf))  # Output will be 0 or 1 (or True/False)


[0]


In [ ]:
# 1. Preprocess the sample
cleaned_sample = preprocess_text(sample[0])

# 2. Vectorize it using your TF-IDF vectorizer
sample_vector = vectorizer.transform([cleaned_sample])

# 3. Predict with your trained model
prediction = model.predict(sample_vector)

# 4. Output result
print("Prediction:", "FAKE" if prediction[0] == 1 else "REAL")


Prediction: REAL


In [ ]:
import pickle

In [ ]:
with open('Fake_News_Detection.pkl', 'wb') as file:
  pickle.dump(model,file)

In [ ]:
with open('tfidf_vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)


In [ ]:
with open('Fake_News_Detection.pkl', 'rb') as file:
    loaded_model = pickle.load(file)


In [ ]:
with open('tfidf_vectorizer.pkl', 'rb') as file:
    loaded_vectorizer = pickle.load(file)


In [ ]:
def predict_news(news_text):

    cleaned_text = preprocess_text(news_text)

    text_vector = loaded_vectorizer.transform([cleaned_text])

    prediction = loaded_model.predict(text_vector)

    return "Fake News" if prediction[0] == 1 else "Real News"


In [ ]:
example_news = "The government has announced a new COVID-19 relief fund."
print(predict_news(example_news))


Real News


In [ ]:
from google.colab import files

files.download('Fake_News_Detection.pkl')
files.download('tfidf_vectorizer.pkl')  # if you also want to download vectorizer


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>